In [ ]:
import pymongo
import pandas as pd
import numpy as np
import folium
import requests
from bs4 import BeautifulSoup as bs
from pandas.io.json import json_normalize
from pymongo import MongoClient
import re
import textract
import matplotlib.pyplot as plt
client=MongoClient()
db=client.DB

In [ ]:
urljll = 'https://www.us.jll.com/en/research/cities-research/innovation-geographies'
urlnumbeo = 'https://www.numbeo.com/cost-of-living/rankings.jsp'

In [ ]:
htmlinta = requests.get(urljll).content
soupinta = bs(htmlinta,'html.parser')
jllcity = [c for c in soupinta.find_all('ul',{'class':'list-cities'})]
most_innovative_city = jllcity[0].text.strip().split('\n')
most_talented_city = jllcity[1].text.strip().split('\n')
innovation_city = []
for y in most_innovative_city:
    t = [i for j in re.findall('([A-Z][a-z]+ ?[A-Z][a-z]+)|([A-Z][a-z]+)',y) for i in j if i != '']
    innovation_city.append(t[0])
talent_city = []
for y in most_talented_city:
    t = [i for j in re.findall('([A-Z][a-z]+ ?[A-Z][a-z]+)|([A-Z][a-z]+)',y) for i in j if i != '']
    talent_city.append(t[0])

In [ ]:
htmlcost = requests.get(urlnumbeo).content
soupcost = bs(htmlcost,'html.parser')
citycostname = [e.text.split(',')[0] for e in soupcost.find_all('td',{'class':'cityOrCountryInIndicesTable'})]
citycostindexes = [e.text for e in soupcost.find_all('td',{'style':'text-align: right'})]
citycostindex = []
counter = 0
for i in range(len(citycostindexes)):
    try:
        citycostindex.append(float(citycostindexes[counter]))
        counter += 6
    except:
        break
citycost = dict(zip(citycostname,citycostindex))

In [ ]:
cities_research = set(innovation_city+talent_city)

In [ ]:
indexpais = pd.DataFrame(cities_research)
indexpais.columns = ['ciudad']

In [ ]:
dicic = {r:s+1 for r,s in zip(innovation_city,range(len(innovation_city)))}

In [ ]:
dictc = {r:s+1 for r,s in zip(talent_city,range(len(talent_city)))}

In [ ]:
diccc =  {'Austin':{'latitude':30.267153,'longitude':-97.743057},
          'Berlin':{'latitude':52.520008,'longitude':13.404954},
          'Boston':{'latitude':42.360081,'longitude':-71.058884},
          'Paris':{'latitude':48.856613,'longitude':2.352222},
          'Shenzhen':{'latitude':22.543097,'longitude':114.057861},
          'San Jose':{'latitude':37.338207,'longitude':-121.886330},
          'Munich':{'latitude':48.135124,'longitude':11.581981},
          'Helsinki':{'latitude':60.169857,'longitude':24.938379},
          'Singapore':{'latitude':1.352083,'longitude':103.819839},
          'Zurich':{'latitude':47.376888,'longitude':8.541694},
          'Toronto':{'latitude':43.653225,'longitude':-79.383186},
          'Amsterdam':{'latitude':52.370216,'longitude':4.895168},
          'Edinburgh':{'latitude':55.953251,'longitude':-3.188267},
          'Los Angeles':{'latitude':34.052235,'longitude':-118.243683},
          'Shanghai':{'latitude':31.230391,'longitude':121.473701},
          'Sydney':{'latitude':-33.868820,'longitude':151.209290},
          'New York':{'latitude':40.712776,'longitude':-74.005974},
          'Stockholm':{'latitude':59.329323,'longitude':18.068581},
          'London':{'latitude':51.507351,'longitude':-0.127758},
          'Denver':{'latitude':39.739235,'longitude':-104.990250},
          'Melbourne':{'latitude':-37.813629,'longitude':144.963058},
          'Tokyo':{'latitude':35.689487,'longitude':139.691711},
          'San Francisco':{'latitude':37.774929,'longitude':-122.419418},
          'Seattle':{'latitude':47.606209,'longitude':-122.332069},
          'Oslo':{'latitude':59.913868,'longitude':10.752245},
          'Beijing':{'latitude':39.904202,'longitude':116.407394},
          'Seoul':{'latitude':37.5665,'longitude':126.9780},
          'Washington':{'latitude':38.9072,'longitude':-77.0369}}

In [ ]:
incity = []
for t in indexpais.ciudad:
    if t in dicic:
        incity.append(dicic[t])
    else:
        incity.append(0)

talcity = []
for t in indexpais.ciudad:
    if t in dictc:
        talcity.append(dictc[t])
    else:
        talcity.append(0)
        
coordcity = []
for t in indexpais.ciudad:
    if t in diccc:
        coordcity.append(diccc[t])
    else:
        coordcity.append(0)
        
costcity = []
for t in indexpais.ciudad:
    if t in citycost:
        costcity.append(citycost[t])
    else:
        costcity.append(0)        
        
indexpais['rank_innov'] = incity
indexpais['rank_talento'] = talcity
indexpais['rank_costo'] = costcity
indexpais['coordenadas'] = coordcity

In [ ]:
indexpais = indexpais.assign(integral=np.where(indexpais.rank_innov>0,25-indexpais.rank_innov*(15)/19,0)+
                          np.where(indexpais.rank_talento>0,25-indexpais.rank_talento*(15)/19,0)+
                          (2134.5/indexpais.rank_costo))

In [ ]:
indexpais.set_index('ciudad',inplace=True)

In [ ]:
indexpais.sort_values('integral',ascending=False)

In [ ]:
z = db.db.find({'number_of_employees':{'$lte':1000},'founded_year':{'$gte':2000},'ipo':None, 'offices.0.city':{'$exists': True },'offices.0.latitude':{'$exists': True }},{'_id':0,'name':1,'category_code':1,'number_of_employees':1,'offices':1})
zz = pd.DataFrame(z)
zz = zz.assign(city=zz.offices)
zz.city = zz.city.apply(lambda x : x[0]['city'])

In [ ]:
def get_first(data):
    res=[]
    ofi=[]
    data=data['offices']
    for e in data:
        principal=None   # solo las tienen geodata
        if e[0]['latitude'] and e[0]['longitude']:
            principal={'type':'Point',
                       'coordinates':[
                           e[0]['longitude'],
                           e[0]['latitude']
                       ]}
            
        ofi.append(principal)
        
        res.append({
            'total_offices':len(e),
            'lat':e[0]['latitude'],
            'lng':e[0]['longitude'],
            'oficina_principal':principal
        })
    
    return ofi

In [ ]:
zz = zz.assign(geo=get_first(zz))

In [ ]:
zz = zz.assign(lat = zz.offices.apply(lambda x : x[0]['latitude']))

In [ ]:
zz = zz.assign(long = zz.offices.apply(lambda x : x[0]['longitude']))

In [ ]:
zz

In [ ]:
choice_ciudad = input()

In [ ]:
mapa=folium.Map(location=[indexpais.loc[choice_ciudad].coordenadas['latitude'],indexpais.loc[choice_ciudad].coordenadas['longitude']], tiles='stamentoner', zoom_start=12)
folium.CircleMarker([indexpais.loc[choice_ciudad].coordenadas['latitude'], indexpais.loc[choice_ciudad].coordenadas['longitude']], radius=10, icon=folium.Icon()).add_to(mapa)

In [ ]:
startupfilter = zz[zz.city==choice_ciudad]
startupfilter

In [ ]:
for i in range(len(startupfilter.lat)):
    try:
        folium.Marker([startupfilter.lat.iloc[i],startupfilter.long.iloc[i]],popup = startupfilter.name.iloc[i],tooltip='haz click aqui').add_to(mapa)
    except:
        pass
mapa

In [ ]:
db.startupfilter.create_index([('geo', '2dsphere')])

In [ ]:
def find_near(geopoint, radio=100):
    return db.startupfilter.find({
        'geo':{'$near':{'$geometry':geopoint,'$maxDistance':radio} 
        }
    }
    )

In [ ]:
refcity={'type':'Point', 'coordinates':[indexpais.loc[choice_ciudad].coordenadas['latitude'],
                                        indexpais.loc[choice_ciudad].coordenadas['longitude']]}
radio = 100

In [ ]:
refcity

In [ ]:
n_ofi=find_near(refcity, radio).count()

print ('Hay {} oficinas a {} metros'.format(n_ofi, radio))
print ('La mas cercana es {}'.format(list(find_near(refcity, radio).limit(1))[0]['name']))